<a href="https://colab.research.google.com/github/jonkrohn/DLTFpT/blob/master/notebooks/cartpole_dqn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cartpole Deep Q-learning Network

Deep Q-Learning Network z użyciem bibliotek Keras i OpenAI Gym, bazujący na [kodzie Keona Kima](https://github.com/keon/deep-q-learning/blob/master/dqn.py).

#### Importujemy zależności

In [1]:
import random
import gym
import numpy as np
from collections import deque
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Nadam
import os

In [2]:
pip install gym

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import matplotlib.pyplot as plt
%matplotlib inline
from IPython import display

In [4]:
!pip install pyglet==1.5.15 gym==0.18.3 pyvirtualdisplay==2.2

from IPython.display import display, clear_output
import ipywidgets as widgets
from PIL import Image
from time import sleep
from IPython.display import display
from IPython.display import clear_output
import io
import sys

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached pyglet-1.5.15-py3-none-any.whl (1.1 MB)
  Using cached gym-0.18.3.tar.gz (1.6 MB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [5]:
import tensorflow as tf

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

#### Ustawiamy hiperparametry

In [6]:
env = gym.make('CartPole-v0') # inicjalizujemy środowisko

/usr/local/lib/python3.10/dist-packages/gym/envs/registration.py:593: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [7]:
state_size = env.observation_space.shape[0]
state_size

4

In [8]:
action_size = env.action_space.n
action_size

2

In [9]:
batch_size = 32

In [10]:
n_episodes = 1000 # ile razy agent ma zagrać w grę

In [11]:
output_dir = 'model_output/cartpole/' #lokalizacja wyjściowa dla pliku z wagami

In [12]:
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

#### Definiuejmy klasę agenta (bardzo ważne są komentarze)

In [13]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size # liczba rodzajów informacji o stanach, tutaj: położenie i prędkość wózka, kąt odchylenia i prędkość kątowa tyczki
        self.action_size = action_size # liczba możliwych czynności do wykonania, tutaj dwie: ruch w lewo, ruch w prawo
        self.memory = deque(maxlen=2000) # struktura będąca kolejką o dwóch końcach; działa jak lista, ale elementy mogą być dodawane i usuwane z obu stron, przechowuje 2000 elementów, po dojściu 2001-ego pierwszy zostaje usunięty
        self.gamma = 0.95 # gamma, czynnik dyskontowy: pozwala agentowi brać pod uwagę przyszłe akcje w zestawieniu z natychmiastowymi
        self.epsilon = 1.0 # współczynnik eksploracji: stosunek liczby czynności wykonywanych losowo do tych wykonywanych zgodnie z "wiedzą" zgromadzoną przez sieć, na początku 100%, bo nie posiadamy żadnej wiedzy
        self.epsilon_decay = 0.995 # współczynnik przez który po każdym epizodzie przemnażamy epsilon tak, aby agent stopniowo działał coraz mniej losowo a coraz bardziej korzystał z wiedzy
        self.epsilon_min = 0.01 # mnimalna wartość epsilona
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self): # budujemy sieć do aproksymacji funkcji wartości Q:
        model = Sequential()
        model.add(Dense(32, activation='relu', input_dim=self.state_size)) # pierwsza warstwa ukryta, stan jako wejście
        model.add(Dense(32, activation='relu')) # druga warstwa ukryta
        model.add(Dense(self.action_size, activation='linear')) # dwie możliwe akcje, a więc 2 neurony: 0 i 1 (Lewo/Prawo)
        model.compile(loss='mse', optimizer=Nadam(learning_rate=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done): # metoda która rejestruje przebieg gry
        self.memory.append((state, action, reward, next_state, done)) # lista poprzednich doświadczeń (bieżący stan, czynność, nagroda, następny stan, flaga logiczna done - True tylko jeśli dany krok jest ostatnim w danym epizodzie)

    def train(self, batch_size): # metoda która trenuje sieć zgodnie z wiedzą posiadaną w strukturze memory
        minibatch = random.sample(self.memory, batch_size) # pobierz minibatch ze struktury memory
        for state, action, reward, next_state, done in minibatch:
            target = reward # jeśli done (epizod skończony), wtedy zastąp target nagrodą
            if not done: # jeśli nie skończony, to prognozuj przyszłą zdyskontowaną nagrodę
                target = (reward + self.gamma * np.amax(self.model.predict(next_state)[0]))
                          # (target) = reward + gamma * (maksymalna docelowa Q osiagana przez akcję a')
            target_f = self.model.predict(state) # oszacowanie zdyskontowanej nagrody dla aktualnego stanu
            target_f[0][action] = target # wynik target_f zastępowany jest nagrodą, linijka 30 lub 28
            self.model.fit(state, target_f, epochs=1, verbose=0) # pojedyncza epoka treningu gdzie daną wejściową jest stan, a daną wyjściową target_f, która zawiera oszacowanie maksymalnej zdyskontowanej przyszłej nagrody; uczenie, czyli zmiana parametrów, zmniejsza koszt pomiędzy szacowaną a prawdziwą nagrodą, pozwala dokładniej prognozować jakie czynności z większym prawdopodobieństwem przyczynią się do uzyskania maksymalnej nagrody w danym stanie
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def act(self, state): # metoda która dokonuje wyboru czynności
        if np.random.rand() <= self.epsilon: # jeśli działasz losowo, wybierz losową akcję
            return random.randrange(self.action_size)
        act_values = self.model.predict(state) # jeśli nie działasz losowo, prognozuj wartość nagrody bazując na aktualnym stanie
        return np.argmax(act_values[0]) # wybierz akcję która da największą nagrodę

    def save(self, name):
        self.model.save_weights(name)

    def load(self, name):
        self.model.load_weights(name)

#### Interakcja ze środowiskiem

In [14]:
agent = DQNAgent(state_size, action_size) # inicjalizuj agenta

In [15]:
def render_env(env, fps=30):
    clear_output(wait=True)
    img = env.render(mode="rgb_array")
    img_pil = Image.fromarray(img)
    img_byte_arr = io.BytesIO()
    img_pil.save(img_byte_arr, format='PNG')
    img_widget = widgets.Image(value=img_byte_arr.getvalue())
    display(img_widget)
    sleep(1 / fps)

def display_direction(action):
    if action == 0:
        direction = "Left"
    else:
        direction = "Right"
    print(f"Cart is moving: {direction}")



**Zadanie za 3 punkty: dodaj odpowiednie funkcje, żeby wyświetlać w którą stronę przesuwa się wózek oraz żeby renderować całe środowisko (filmik poklatkowy)**

In [16]:
env = gym.make("CartPole-v1")

In [17]:
tf.keras.backend.clear_session()
tf.compat.v1.reset_default_graph()

In [18]:
sys.stdout = open(os.devnull, 'w')

In [24]:
def render_env(env, fps=60):
    clear_output(wait=True)
    img = env.render(mode="rgb_array")
    img_pil = Image.fromarray(img)
    img_byte_arr = io.BytesIO()
    img_pil.save(img_byte_arr, format='PNG')
    img_widget = widgets.Image(value=img_byte_arr.getvalue())
    display(img_widget)
    sleep(1 / fps)

In [20]:
def display_direction(action):
    if action == 0:
        direction = "Left"
    else:
        direction = "Right"
    print(f"Cart is moving: {direction}")

In [22]:
tf.keras.backend.clear_session()
tf.compat.v1.reset_default_graph()

In [ ]:
n_episodes = 100

for e in range(n_episodes):
    state = env.reset()
    state = np.reshape(state, [1, state_size])
    done = False
    time = 0  # Initialize the time variable

    while not done:
        render_env(env)  # Render the environment
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        reward = reward if not done else -10
        next_state = np.reshape(next_state, [1, state_size])
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        if done:
            print("episode: {}/{}, score: {}, e: {:.2}"
                  .format(e, n_episodes - 1, time, agent.epsilon))
        time += 1

    if len(agent.memory) > batch_size:
        agent.train(batch_size)
    if e % 50 == 0:
        agent.save(output_dir + "weights_" + '{:04d}'.format(e) + ".hdf5")

    display_direction(action)  # Display the direction of cart movement

env.close()


Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02X\x00\x00\x01\x90\x08\x02\x00\x00\x00\xfdW\x89\xc…

In [ ]:
sys.stdout = sys.__stdout__